In [1]:
sc

org.apache.spark.SparkContext@96a8de99

In [2]:
sc.version

2.1.0

positions-geographiques-des-stations-du-reseau-ratp

In [4]:
// The code was removed by DSX for sharing.

+-------+--------------------+--------------------+--------------------+------------------+------------------+----------+-----------+
|stop_id|           stop_name|           stop_desc|               coord|          stop_lat|          stop_lon|code_INSEE|departement|
+-------+--------------------+--------------------+--------------------+------------------+------------------+----------+-----------+
|   2158|       Achères-Ville|Avenue de Conflan...|48.9700771763, 2....|48.970077176304514|2.0776181820083806|     78005|         78|
|   2159|              Alésia|Place Victor et H...|48.8280660197, 2....| 48.82806601968645| 2.326827420050836|     75114|         75|
|   2172|            Concorde|Tuileries (jardin...|48.8654893909, 2....|  48.8654893908901| 2.321411789213801|     75108|         75|
|   2174|          Convention|Vaugirard (337 ru...|48.8371369496, 2....| 48.83713694956265| 2.296396090155559|     75115|         75|
|   2178|Courcelle-sur-Yvette|Rue Fernand Leger...|48.70076301

Validatations

In [48]:
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types._

val customSchema = StructType(Array(
    StructField("JOUR", DateType, true),
    StructField("CODE_STIF_TRNS", StringType, true),
    StructField("CODE_STIF_RES", StringType, true),
    StructField("CODE_STIF_ARRET", StringType, true),
    StructField("LIBELLE_ARRET", StringType, true),
    StructField("ID_REFA_LDA", StringType, true),
    StructField("CATEGORIE_TITRE", StringType, true),
    StructField("NB_VALD", DoubleType, true)))

val dfValidation = spark.
    read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
    option("header", "true").
    schema(customSchema).
    option("delimiter",";").
    option("mode", "DROPMALFORMED").
    load("swift://sncf." + name + "/validations-sur-le-reseau-ferre-nombre-de-validations-par-jour-1er-semestre-2015.csv")
dfValidation.show(5)
val nbrow = dfValidation.count

+----------+--------------+-------------+---------------+--------------------+-----------+---------------+-------+
|      JOUR|CODE_STIF_TRNS|CODE_STIF_RES|CODE_STIF_ARRET|       LIBELLE_ARRET|ID_REFA_LDA|CATEGORIE_TITRE|NB_VALD|
+----------+--------------+-------------+---------------+--------------------+-----------+---------------+-------+
|2016-01-05|           100|          110|            762|SAINT-DENIS-PORTE...|      72285|      AMETHYSTE|  264.0|
|2016-01-05|           100|          110|            766|       SAINT-GEORGES|      71366|            TST|  103.0|
|2016-01-05|           100|          110|            768|       SAINT-JACQUES|      71041|            TST|  415.0|
|2016-01-05|           100|          110|            769|        SAINT-LAZARE|      71370|      IMAGINE R|15406.0|
|2016-01-05|           100|          110|            777|SAINT-PAUL (LE MA...|      71222|    AUTRE TITRE|   88.0|
+----------+--------------+-------------+---------------+--------------------+--

Name: java.lang.InterruptedException
Message: null
StackTrace:   at java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1009)
  at java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1315)
  at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:202)
  at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:218)
  at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:153)
  at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:620)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:1931)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:1944)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:1957)
  at org.apache.spark.SparkContext.runJob(SparkContext.scala:1971)
  at org.apache.spark.rdd.RDD$$anonfun$collect$1.apply(RDD.scala:954)
  at org.apache.spark.rdd.RDDOperationScope$.

## 1. Total, min, max, écart type de validations sur l'ensemble du réseau

In [15]:
dfValidation.printSchema()

root
 |-- JOUR: date (nullable = true)
 |-- CODE_STIF_TRNS: string (nullable = true)
 |-- CODE_STIF_RES: string (nullable = true)
 |-- CODE_STIF_ARRET: string (nullable = true)
 |-- LIBELLE_ARRET: string (nullable = true)
 |-- ID_REFA_LDA: string (nullable = true)
 |-- CATEGORIE_TITRE: string (nullable = true)
 |-- NB_VALD: double (nullable = true)



In [23]:
// Register the DataFrame as a global temporary view
//dfValidation.createGlobalTempView("validation")
spark.sql("desc global_temp.validation").show()


+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|           JOUR|     date|   null|
| CODE_STIF_TRNS|   string|   null|
|  CODE_STIF_RES|   string|   null|
|CODE_STIF_ARRET|   string|   null|
|  LIBELLE_ARRET|   string|   null|
|    ID_REFA_LDA|   string|   null|
|CATEGORIE_TITRE|   string|   null|
|        NB_VALD|   double|   null|
+---------------+---------+-------+



In [51]:
import org.apache.spark.sql.functions._
val result = spark.sql("select MAX(NB_VALD) as maxi from global_temp.validation")
result.show()